<a href="https://colab.research.google.com/github/RaphaelCarvalh/BootCampAVANTI_machine_learning/blob/ativ04-et01-analise-dataset/Et01_analise_dataset_arvore_decis%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto Clothing Co-Parsing - Etapa 1 - Notebook: Análise do Dataset - TIV-04-ET-02


In [ ]:
import pandas as pd

# Carregar dataset já limpo
df = pd.read_csv("df_clean.csv")
print(df.head())
print(f"Total de imagens: {len(df)}")

transformações e augmentations

In [ ]:
from torchvision import transforms

print("Configurando transformações e augmentations...\n")

IMG_SIZE = 224  # definir tamanho das imagens

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("Transformações configuradas com sucesso.")

Configurando transformações e augmentations...

Transformações configuradas com sucesso.


Criar Datasets e DataLoaders

In [ ]:
from torch.utils.data import Dataset, DataLoader
import cv2
import torch
from sklearn.model_selection import train_test_split

# Pipeline para carregar imagens
class FlowerDataset(Dataset):
    def __init__(self, data, transform):
        """
        data: array ou DataFrame com ['images', 'labels']
        transform: transformações de imagem definidas (train_transform ou val_transform)
        """
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx, 0], self.data[idx, 1]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform:
            img = self.transform(img)
        return img, label


Dividir os dados em treino, validação e teste

In [ ]:
# Divisão: 80% treino, 10% validação, 10% teste
train_data, temp_data = train_test_split(df.values, test_size=0.2, random_state=42, stratify=df.values[:,1])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data[:,1])

# Criar datasets
train_ds = FlowerDataset(train_data, transform=train_transform)
val_ds = FlowerDataset(val_data, transform=val_test_transform)
test_ds = FlowerDataset(test_data, transform=val_test_transform)

# Criar dataloaders
BATCH_SIZE = 32  # Podemos ajustar depois se a memória permitir

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Treino: {len(train_ds)} | Validação: {len(val_ds)} | Teste: {len(test_ds)}")


Treino: 1678 | Validação: 210 | Teste: 210


criar os DataLoaders

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32  # podemos ajustar depois se precisar

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train batches: {len(train_dl)}, Val batches: {len(val_dl)}, Test batches: {len(test_dl)}")

Train batches: 53, Val batches: 7, Test batches: 7
